# Standard Errors

In [1]:
import numpy as np
import math 
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import scipy.stats as stats
from sklearn import datasets, linear_model

import statsmodels.regression.linear_model as lm
import statsmodels.api as sm

def SE(option):
    alpha = 0.5
    beta = 2
    N = 300
    regr = linear_model.LinearRegression()
    
    X = np.linspace(-3, 3, N)
    Y1 = [alpha + beta * xx + np.random.normal(0, 2) for xx in X]
    Y2 = [alpha + beta * xx + np.random.normal(0, 2) * (xx+3)/3 for xx in X]    
    X = np.array(X)
    Y1 = np.array(Y1)
    Y2 = np.array(Y2)

    fig = plt.figure(figsize=(20, 7))    
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)   
    



    cov = np.zeros((300, 300))
    for i in range(300):
        cov[i][i] = 1
    Yhomo = np.random.multivariate_normal(np.zeros(300), cov)

    xx = np.random.normal(0, 1, 300)
    yy = [alpha + x * beta + y for x, y in zip(xx, Yhomo)]
    regr.fit(np.array(xx).reshape(N, 1), np.array(yy).reshape(N, 1))
    yyy = [regr.intercept_ + x * regr.coef_[0] for x in xx]
    residuals = [y - (regr.intercept_ + x * regr.coef_[0]) for x, y in zip(xx, yy)]
    betahomo = regr.coef_[0][0]
    sehomo = 1 / sum([(x - np.mean(xx))**2 for x in xx])

    if option == 'Homoscedastic':    
        ax1.plot(xx, yy, 'o')
        ax1.plot(xx, yyy, label='fitted line')
       
        ax2.plot(residuals, 'o')
        ax2.axhline(y=0, c='C1', label='residual=0')
        
        ax1.set_xlabel('X')
        ax1.set_ylabel('Y')
        ax1.set_title('Scatterplot of homoscedastic data')
        ax2.set_xlabel('X')
        ax2.set_ylabel('residual')
        ax2.set_title('Residual plot of homoscedastic data')
        ax1.legend()
        ax2.legend()
        
        plt.show()

        xx = sm.add_constant(xx)
        model = lm.OLS(yy, xx)
        results = model.fit()
        #print (results.params, results.bse)
        print ("estimated betahat (OLS): " + str(results.params[1]))
        print("standard error (OLS): " + str(results.bse[1]))
        results2 = model.fit(cov_type='HC0')
        mygroups = [0]*100 + [1]*100 + [2]*100
        results3 = model.fit(cov_type='cluster', cov_kwds={'groups': mygroups})
        



    cov = np.zeros((300, 300))
    for i in range(300):
        cov[i][i] = (i+1)/150.0
    Yhetero = np.random.multivariate_normal(np.zeros(300), cov) #increasing variance by index

    #xx = np.linspace(-1, 1, 300)
    xx = sorted(np.random.normal(0, 1, 300))
    yy = [alpha + x * beta + y for x, y in zip(xx, Yhetero)]
    #yyy = [alpha + x * beta for x in xx]
    regr.fit(np.array(xx).reshape(N, 1), np.array(yy).reshape(N, 1))
    yyy = [regr.intercept_ + x * regr.coef_[0] for x in xx]        
    residuals = [y - (regr.intercept_ + x * regr.coef_[0]) for x, y in zip(xx, yy)]
    betahetero = regr.coef_[0][0]
    sehetero = 1 / np.dot(xx, xx) * sum([resid**2 * x**2 for resid, x in zip(residuals, xx)]) * 1 / np.dot(xx, xx)
    sehetero = sehetero[0]

    if option == 'Heteroscedastic': 
        ax1.plot(xx, yy, 'o')
        ax1.plot(xx, yyy, label='fitted line')
        ax2.plot(residuals, 'o')
        ax2.axhline(y=0, c='C1', label='residual=0')
        ax1.set_xlabel('X')
        ax1.set_ylabel('Y')
        ax1.set_title('Scatterplot of heteroscedastic data')
        ax2.set_xlabel('X')
        ax2.set_ylabel('residual')
        ax2.set_title('Residual plot of heteroscedastic data')
        ax1.legend()
        ax2.legend()
        plt.show()

        xx = sm.add_constant(xx)
        model = lm.OLS(yy, xx)
        results = model.fit()
        #print (results.params, results.bse)
        results2 = model.fit(cov_type='HC0')
        #print (results2.params, results2.bse)
        mygroups = [0]*100 + [1]*100 + [2]*100
        results3 = model.fit(cov_type='cluster', cov_kwds={'groups': mygroups})
        #print (results3.params, results3.bse)
        print ("estimated betahat (OLS): " + str(results.params[1]))
        print ("standard error (OLS): " + str(results.bse[1]))
        print ("estimated betahat (heteroscedasticity-robust): " + str(results2.params[1]))
        print ("standard error (heteroscedasticity-robust): " + str(results2.bse[1]))

    xx = np.random.normal(0, 1, 300)
    eps1 = np.random.normal(0, np.random.rand()*1, 100)
    eps2 = np.random.normal(0, np.random.rand()*3, 100)
    eps3 = np.random.normal(0, np.random.rand()*5, 100)
    Y1 = [alpha + beta*x + e for x, e in zip(xx[:100], eps1)]
    Y2 = [alpha + beta*x + e for x, e in zip(xx[100:200], eps2)]        
    Y3 = [alpha + beta*x + e for x, e in zip(xx[:200], eps3)]
    yy = np.hstack((Y1, Y2, Y3))
    regr.fit(np.array(xx).reshape(N, 1), yy.reshape(N, 1))
    yyy = [regr.intercept_ + x * regr.coef_[0] for x in xx]  
    r1 = [y - (regr.intercept_ + x * regr.coef_[0][0]) for x, y in zip(xx, Y1)]
    r2 = [y - (regr.intercept_ + x * regr.coef_[0][0]) for x, y in zip(xx, Y2)]
    r3 = [y - (regr.intercept_ + x * regr.coef_[0][0]) for x, y in zip(xx, Y3)]
    residuals = np.vstack((r1, r2, r3))

    middleterm = 0
    for i in range(3):
        middleterm += np.vdot(xx[100*i:100*(i+1)], residuals[100*i:100*(i+1)])**2

    betacluster = regr.coef_[0][0]
    secluster = 1 / np.dot(xx, xx) * middleterm * 1 / np.dot(xx, xx)

    if option == "Cluster-Robust":
        ax1.plot(xx[:100], Y1, 'bo', label='cluster1')
        ax1.plot(xx[100:200], Y2, 'ro', label='cluster2')
        ax1.plot(xx[200:], Y3, 'go', label='cluster3')
        ax1.plot(xx, yyy, 'C1', label='fitted line')
        ax1.set_xlabel('X')
        ax1.set_ylabel('Y')
        ax1.set_title('Scatterplot of clustered data')
        ax1.legend()
        

        ax2.plot(r1, 'bo', label='cluster1')
        ax2.plot(r2, 'ro', label='cluster2')
        ax2.plot(r3, 'go', label='cluster3')
        ax2.axhline(y=0, c='C1', label='residual=0')
        ax2.set_xlabel('X')
        ax2.set_ylabel('residual')
        ax2.set_title('Residual plot of clustered data')
        ax2.legend()
        plt.plot()
        plt.show()

        xx = sm.add_constant(xx)
        model = lm.OLS(yy, xx)
        results = model.fit()
        #print (results.params, results.bse)
        results2 = model.fit(cov_type='HC0')
        #print (results2.params, results2.bse)
        mygroups = [0]*100 + [1]*100 + [2]*100
        results3 = model.fit(cov_type='cluster', cov_kwds={'groups': mygroups})
        #print (results3.params, results3.bse)        

        print ("estimated betahat (OLS): " + str(results.params[1]))
        print ("standard error (OLS): " + str(results.bse[1]))
        print ("estimated betahat (heteroscedasticity-robust): " + str(results2.params[1]))
        print ("standard error (heteroscedasticity-robust): " + str(results2.bse[1]))
        print ("estimated betahat (cluster-robust): " + str(results3.params[1]))
        print ("standard error (cluster-robust): " + str(results3.bse[1]))

interact_manual(SE, option=widgets.Dropdown(options=['Homoscedastic', 'Heteroscedastic', 'Cluster-Robust']))

C:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<function __main__.SE>